# Quick Test Notebook - Dividend Capture Algorithm

**Purpose:** Fast iteration testing during development

**Runtime:** ~2-3 minutes

---

## Setup (Run Once)

In [ ]:
# Clone and setup
!git clone https://github.com/DennisPoniros/XdividendALG.git
%cd /content/XdividendALG
!pip install -q pandas numpy scipy yfinance matplotlib seaborn statsmodels scikit-learn alpaca-trade-api

print("✅ Setup complete")

## 🔄 Refresh Code (Run this to pull latest changes)

In [ ]:
# Pull latest code from main branch
%cd /content/XdividendALG
!git pull origin main

# Reload Python modules
import importlib
import sys

modules_to_reload = ['config', 'data_manager', 'strategy', 'backtester', 'risk_manager', 'analytics', 'logger']
for mod in modules_to_reload:
    if mod in sys.modules:
        importlib.reload(sys.modules[mod])

print("✅ Code refreshed")

## Test Level 1: Validation (30 days, ~1 min)

In [ ]:
from backtester import Backtester
from config import validate_config

# Validate config
validation = validate_config()
if validation['errors']:
    print("❌ Config errors:")
    for e in validation['errors']:
        print(f"  - {e}")
else:
    print("✅ Config valid")

# Quick 30-day test
print("\n🧪 Running 30-day validation...")
bt = Backtester(
    start_date='2024-10-01',
    end_date='2024-10-31',
    initial_capital=50_000
)

results = bt.run_backtest(mode='full')

if results:
    print(f"\n{'='*60}")
    print(f"✅ Total Trades: {results.get('total_trades', 0)}")
    print(f"✅ Win Rate: {results.get('win_rate_pct', 0):.1f}%")
    print(f"✅ Sharpe: {results.get('sharpe_ratio', 0):.2f}")
    print(f"✅ Return: {results.get('total_return_pct', 0):.2f}%")
    print(f"✅ Max Drawdown: {results.get('max_drawdown_pct', 0):.2f}%")
    print(f"{'='*60}")
    
    # Cache stats
    cache = bt.dm.get_cache_stats()
    print(f"\n📊 API Efficiency: {cache['hit_rate_pct']:.1f}% cache hit rate")
    print(f"📊 API Calls: {cache['api_calls']}")
else:
    print("⚠️  No results - check for errors above")

## Test Level 2: Medium Test (3 months, ~3 min)

In [ ]:
print("🧪 Running 3-month test...")
bt2 = Backtester(
    start_date='2024-08-01',
    end_date='2024-10-31',
    initial_capital=100_000
)

results2 = bt2.run_backtest(mode='full')

if results2:
    print(f"\n{'='*60}")
    print(f"📊 3-MONTH RESULTS")
    print(f"{'='*60}")
    print(f"Total Trades: {results2.get('total_trades', 0)}")
    print(f"Win Rate: {results2.get('win_rate_pct', 0):.1f}%")
    print(f"Sharpe: {results2.get('sharpe_ratio', 0):.2f}")
    print(f"Total Return: {results2.get('total_return_pct', 0):.2f}%")
    print(f"Max Drawdown: {results2.get('max_drawdown_pct', 0):.2f}%")
    print(f"Avg Trade: {results2.get('avg_return_pct', 0):.2f}%")
    print(f"{'='*60}")

## Test Level 3: Full Year (run when stable, ~10 min)

In [ ]:
# Only run this when you're confident in changes
print("🚀 Running full year test...")
bt3 = Backtester(
    start_date='2024-01-01',
    end_date='2024-10-31',
    initial_capital=100_000
)

results3 = bt3.run_backtest(mode='full')

if results3:
    # Generate full analytics
    from analytics import create_performance_report
    import os
    
    os.makedirs('/content/outputs', exist_ok=True)
    plots, html_report = create_performance_report(results3, output_dir='/content/outputs')
    
    print(f"\n{'='*60}")
    print(f"📊 FULL YEAR RESULTS")
    print(f"{'='*60}")
    print(f"Total Trades: {results3.get('total_trades', 0)}")
    print(f"Win Rate: {results3.get('win_rate_pct', 0):.1f}%")
    print(f"Sharpe: {results3.get('sharpe_ratio', 0):.2f}")
    print(f"Total Return: {results3.get('total_return_pct', 0):.2f}%")
    print(f"Max Drawdown: {results3.get('max_drawdown_pct', 0):.2f}%")
    print(f"\n📁 Full report: {html_report}")
    print(f"📁 Plots generated: {len(plots)}")

## 🎯 Quick Checks

Run these cells individually to test specific components

In [ ]:
# Test data fetching
from data_manager import DataManager

dm = DataManager()
prices = dm.get_stock_prices('AAPL', '2024-10-01', '2024-10-31')
print(f"✅ Fetched {len(prices)} price bars for AAPL")
print(prices.tail())

In [ ]:
# Test dividend calendar
div_cal = dm.get_dividend_calendar('2024-10-01', '2024-10-31')
print(f"✅ Found {len(div_cal)} dividend events")
print(div_cal.head(10))

In [ ]:
# Test strategy signal generation
from strategy import DividendCaptureStrategy

strat = DividendCaptureStrategy(dm)
screened = dm.screen_stocks(div_cal, '2024-10-15')
print(f"✅ Screened candidates: {len(screened)}")
if len(screened) > 0:
    print(screened[['ticker', 'ex_date', 'days_to_ex_div', 'quality_score']].head())